# CONFIGURACIONES

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import datetime
from unidecode import unidecode
from fuzzywuzzy import fuzz, process
import numpy as np
import re
import sqlalchemy.exc as sa_exc
from sqlalchemy.exc import SQLAlchemyError
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import unicodedata

## VISUALIZACIÓN

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

# EXTRACT

In [54]:
df = pd.read_excel(r"C:\Users\aleja\OneDrive\Documents\Oracle - Program\orange\01 OVERALL FINAL DATA PHD.xlsx",
                   sheet_name='RAW DATA', )

# TRANSFORM

## SEPARAR LAS EMPRESAS DE LAS MEDICIONES

In [ ]:
companies = df.iloc[7:, :28] 
companies.columns = companies.iloc[0]
companies = companies[2:].reset_index(drop=True)
companies

In [145]:
scores = df.iloc[:, 27:]
scores = scores.iloc[1:]

## ORGANIZAR EL DF

In [146]:
# Paso 1: Identificar las columnas donde alguna fila contiene 'Addressed criteria'
columnas_a_mover = scores.columns[scores.apply(lambda col: col.astype(str).str.contains('Addressed criteria', na=False)).any()]

# Paso 2: Crear un nuevo DataFrame con estas columnas
addressed_criteria = scores[columnas_a_mover].copy()

# Paso 3: Eliminar estas columnas del DataFrame original
scores.drop(columns=columnas_a_mover, inplace=True)

In [147]:
# Paso 1: Identificar las columnas donde alguna fila contiene 'Use of tools'
columnas_a_mover = scores.columns[scores.apply(lambda col: col.astype(str).str.contains('Use of tools', na=False)).any()]

# Paso 2: Crear un nuevo DataFrame con estas columnas
use_tools = scores[columnas_a_mover].copy()

# Paso 3: Eliminar estas columnas del DataFrame original
scores.drop(columns=columnas_a_mover, inplace=True)

### TRANSPONER EL DF

In [148]:
# Eliminar las filas 1, 4 y 7
scores = scores.reset_index(drop=True).drop(index=[1, 4, 7]).reset_index(drop=True)

### CONCATENARLE EL CÓD DE LA EMPRESA

In [105]:
concat_companies = companies[['CODE']]

In [106]:
# Crear 5 filas nulas con las mismas columnas que el DataFrame original
filas_nulas = pd.DataFrame([[None] * concat_companies.shape[1]] * 5, columns=concat_companies.columns)

# Concatenar las filas nulas con el DataFrame original
concat_companies = pd.concat([filas_nulas, concat_companies], ignore_index=True)

In [149]:
scores = pd.concat([concat_companies,scores],axis=1)

In [ ]:
# scores.drop(columns=['CODE'],inplace=True)

In [142]:
# Paso 1: Transponer el DataFrame (convertir filas en columnas)
dfT = scores.T.reset_index(drop=True)

# Paso 2: Renombrar las columnas para que representen los niveles de jerarquía
dfT.columns = ["Dynamic_Capability_Stage", "Business_Parameters", "Business_Sub", "Metrics","Question"] + [f"Valor_{i}" for i in range(len(dfT.columns)-5)]

# Paso 3: Convertir a formato largo (melt)
df_long = dfT.melt(id_vars=["Dynamic_Capability_Stage", "Business_Parameters", "Business_Sub", "Metrics","Question"], 
                   var_name="Medicion", value_name="Valor")

# Paso 4: Eliminar la columna de "Medicion" ya que no aporta información
df_long = df_long.drop(columns=["Medicion"])

In [151]:
# Paso 2: Asegurar que la primera fila sea los nombres de columna
dfT.columns = dfT.iloc[0]  # Asignar la primera fila como nombres de columnas
dfT = dfT.iloc[1:].reset_index(drop=True)  # Eliminar la fila de encabezado y resetear índice

In [153]:
dfT.to_excel("transpuesta.xlsx", index=False)

# VERSIÓN DF PCA

In [177]:
df_combinado= scores.drop(columns='CODE').iloc[:4].apply(lambda x: '_'.join(x.astype(str)), axis=0).reset_index(drop=True)

In [160]:
df_preguntas =  scores.drop(columns='CODE').iloc[4].reset_index(drop=True).to_frame(name="Question")
df_preguntas["Qst_cod"] = df_preguntas.index + 1

In [161]:
df_preguntas

,Question,Qst_cod
0,"Indicate how often your business identifies and defines performance challenges or opportunities, particularly in relation to customer acquisition.",1
1,"Indicate how effective your business is to identify and define challenges or performance opportunities, particularly in relation to customer acquisition.",2
2,"Indicate how often you participate in the identification and definition of challenges or opportunities for performance within your business development processes, particularly in relation to the acquisition of customers.",3
3,"Indicate the typical level of certainty that you experience by identifying and defining challenges or opportunities for business development, particularly in relation to the acquisition of clients.",4
4,Indicate how often your business establishes clear objectives and performance priorities for the acquisition of customers in business development.,5
...,...,...
139,"Please indicate the typical level of certainty you experience when developing optimal management solutions, reconfigurations, and action plans for customer acquisition.",140
140,Indicate how often your company evaluates and monitors management's performance and practices against customer acquisition objectives.,141
141,Indicate how effective your business is in evaluating and monitoring management performance and practices against customer acquisition objectives.,142
142,Indicate how frequently you participate in evaluating and monitoring management practices and performance against customer acquisition objectives.,143


In [197]:
df_valores = scores.drop(columns='CODE').iloc[5:].reset_index(drop=True)

In [179]:
columnas_nuevas =  [f"{df_combinado[i]}_Q{df_preguntas.loc[i,'Qst_cod']}" for i in range(len(df_combinado))]

In [198]:
df_valores.columns = columnas_nuevas

In [199]:
df_valores = pd.concat([companies['CODE'].reset_index(drop=True),df_valores], axis=1)

In [201]:
companies

7,Company,NaN,LT \nLeader of Team)\n \nor\n \nTM \n(Team Member),CODE,Industry,1 industry/sector:,2 company size,3 Age of the company:,NaN,4 Geographic reach location:,5 R&D Expenditure (Research and Development):,6 new products sales:,7 Number of patents/innovations:,8 Annual growth:,9 Last Year Sales,1 status quo,2 Risk take,3 Creative Confidence,4 Questioning,5 Observing,6 Networking,7 Experimenting,8 associating,9 Analysis,10 planning,11 Detail oriented,12 Discipline,"Indicate how often your business identifies and defines performance challenges or opportunities, particularly in relation to customer acquisition."
9,C01,1.0,LT,C01-LT,Manufa,"Manufactura, Moda",Big,29,25+ years,Regional,0.02,17850000000,12,0.18,85000000000,3.463927,2.251553,3.074534,3.314381,4.033684,3.960822,3.91806,2.276159,3.691352,4.444577,3.896082,4.147157,4
10,C01,1.0,TM,C01-TM,Manufa,"Manufactura, Moda",Big,29,25+ years,Regional,0.02,17850000000,12,0.18,85000000000,3.095238,2.380952,2.857143,3.142857,4,3.571429,3.714286,2,3.571429,4.142857,3.714286,3.857143,4
11,C02,2.0,LT,C02-LT,Servic,"Servicios, Comidas y bebidas",Big,6,5-8 years,National,1037000000,2835000000,8,0.24,94500000000,3.940118,2.013458,3.610248,3.742953,4.605112,4.817965,4.060917,4.419016,4.262781,3.873149,3.324654,3.861443,4
12,C02,2.0,TM,C02-TM,Servic,"Servicios, Comidas y bebidas",Big,6,5-8 years,National,1037000000,2835000000,8,0.24,94500000000,3.571429,2.142857,3.392857,3.571429,4.571429,4.428571,3.857143,4.142857,4.142857,3.571429,3.142857,3.571429,4
13,C03,3.0,LT,C03-LT,Manufa,"Manufactura, Moda",Big,8,5-8 years,Local,0,13200000000,4,0.1,110000000000,4.285714,4.285714,5,3.285714,4.571429,4.428571,4.714286,3.857143,3.142857,4.714286,4.285714,5,4
14,C03,3.0,TM,C03-TM,Manufa,"Manufactura, Moda",Big,8,5-8 years,Local,0,13200000000,4,0.1,110000000000,4.047619,5,5,3.285714,5,5,5,4.285714,4.428571,5,3.857143,5,5
15,C04,4.0,LT,C04-LT,Servic,"Servicios, Transporte y logística",Medium,23,15-25 years,International,1200000000,4800000000,6,0.15,13450000000,5,3.571429,5,4.714286,2.857143,3.714286,4.714286,2.571429,4.714286,3.142857,4.571429,3.714286,3
16,C04,4.0,TM,C04-TM,Servic,"Servicios, Transporte y logística",Medium,23,15-25 years,International,1200000000,4800000000,6,0.15,13450000000,4.047619,2.142857,3.928571,3.857143,2,3.285714,4.285714,2.571429,3.714286,2.714286,4.142857,3.714286,4
17,C05,5.0,LT,C05-LT,Servic,"Servicios, Tecnología",Medium,25,15-25 years,National,962500000,3850000000,4,0.08,19250000000,5,3.095238,4.285714,5,3.857143,4.714286,4.142857,4.142857,3.714286,4.714286,4.142857,4.571429,3
18,C05,5.0,TM,C05-TM,Servic,"Servicios, Tecnología",Medium,25,15-25 years,National,962500000,3850000000,4,0.08,19250000000,4.761905,2.142857,4.107143,3.714286,3.857143,3.428571,4,2.714286,3.714286,4.285714,3.571429,3.428571,3


In [200]:
df_valores

,CODE,Sensing_Performance_Business development_Frequency (business)_Q1,Sensing_Performance_Business development_Effectiveness_Q2,Sensing_Performance_Business development_Frequency (individual)_Q3,Sensing_Performance_Business development_Certainty_Q4,Sensing_Performance_Business development_Frequency (business)_Q5,Sensing_Performance_Business development_Effectiveness_Q6,Sensing_Performance_Business development_Frequency (individual)_Q7,Sensing_Performance_Business development_Certainty_Q8,Sensing_Performance_Innovation_Frequency (business)_Q9,Sensing_Performance_Innovation_Effectiveness_Q10,Sensing_Performance_Innovation_Frequency (individual)_Q11,Sensing_Performance_Innovation_Certainty_Q12,Sensing_Performance_Innovation_Frequency (business)_Q13,Sensing_Performance_Innovation_Effectiveness_Q14,Sensing_Performance_Innovation_Frequency (individual)_Q15,Sensing_Performance_Innovation_Certainty_Q16,Sensing_Offer_Value proposal_Frequency (business)_Q17,Sensing_Offer_Value proposal_Effectiveness_Q18,Sensing_Offer_Value proposal_Frequency (individual)_Q19,Sensing_Offer_Value proposal_Certainty_Q20,Sensing_Offer_Value proposal_Frequency (business)_Q21,Sensing_Offer_Value proposal_Effectiveness_Q22,Sensing_Offer_Value proposal_Frequency (individual)_Q23,Sensing_Offer_Value proposal_Certainty_Q24,Sensing_Process and tools_Business development_Frequency (business)_Q25,Sensing_Process and tools_Business development_Effectiveness_Q26,Sensing_Process and tools_Business development_Frequency (individual)_Q27,Sensing_Process and tools_Business development_Certainty_Q28,Sensing_Process and tools_Business development_Frequency (business)_Q29,Sensing_Process and tools_Business development_Effectiveness_Q30,Sensing_Process and tools_Business development_Frequency (individual)_Q31,Sensing_Process and tools_Business development_Certainty_Q32,Sensing_Process and tools_Innovation processes_Frequency (business)_Q33,Sensing_Process and tools_Innovation processes_Effectiveness_Q34,Sensing_Process and tools_Innovation processes_Frequency (individual)_Q35,Sensing_Process and tools_Innovation processes_Certainty_Q36,Sensing_Process and tools_Innovation processes_Frequency (business)_Q37,Sensing_Process and tools_Innovation processes_Effectiveness_Q38,Sensing_Process and tools_Innovation processes_Frequency (individual)_Q39,Sensing_Process and tools_Innovation processes_Certainty_Q40,Sensing_People_Management_Frequency (business)_Q41,Sensing_People_Management_Effectiveness_Q42,Sensing_People_Management_Frequency (individual)_Q43,Sensing_People_Management_Certainty_Q44,Sensing_People_Management_Frequency (business)_Q45,Sensing_People_Management_Effectiveness_Q46,Sensing_People_Management_Frequency (individual)_Q47,Sensing_People_Management_Certainty_Q48,Seizing_Performance_Business development_Frequency (business)_Q49,Seizing_Performance_Business development_Effectiveness_Q50,Seizing_Performance_Business development_Frequency (individual)_Q51,Seizing_Performance_Business development_Certainty_Q52,Seizing_Performance_Business development_Frequency (business)_Q53,Seizing_Performance_Business development_Effectiveness_Q54,Seizing_Performance_Business development_Frequency (individual)_Q55,Seizing_Performance_Business development_Certainty_Q56,Seizing_Performance_Innovation_Frequency (business)_Q57,Seizing_Performance_Innovation_Effectiveness_Q58,Seizing_Performance_Innovation_Frequency (individual)_Q59,Seizing_Performance_Innovation_Certainty_Q60,Seizing_Performance_Innovation_Frequency (business)_Q61,Seizing_Performance_Innovation_Effectiveness_Q62,Seizing_Performance_Innovation_Frequency (individual)_Q63,Seizing_Performance_Innovation_Certainty_Q64,Seizing_Offer_Value proposal_Frequency (business)_Q65,Seizing_Offer_Value proposal_Effectiveness_Q66,Seizing_Offer_Value proposal_Frequency (individual)_Q67,Seizing_Offer_Value proposal_Certainty_Q68,Seizing_Offer_Value proposal_Frequency (business)_Q69,Seizing_Offer_Value proposal_Effectiveness_Q70,Seizing_Offer_